## 네이버뉴스 언론사/일자별 동적 웹크롤링

- 실행 전 라이브러리 설치
- pip install webdriver-manager

- 코드 상단 📌 TARGET_URL 수정해서 그대로 실행
- 예시 URL : https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=009&date=20251218
- "oid=009&date=20251218"처럼 날짜 부분까지 나오도록 페이지 확인 후 입력 (언론사 들어가서 바로 나오는 오늘자 페이지는 날짜 표시가 바로 뜨지 않아서, 다른 일자 선택 후 다시 오늘 일자 선택)
- 👉 "oid=009&date=20251218.csv"로 본 파일과 같은 위치에 저장됨. (oid가 언론사 url 코드)

In [4]:
import time
import pandas as pd  # 데이터프레임 및 CSV 저장을 위해
from datetime import datetime # 파일명 생성용
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ====================================================
# 📌 [크롤링 대상 URL] (여기만 수정해서 쓰세요)
# ====================================================
TARGET_URL = "https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=015&date=20251217"


# ====================================================
# 1. 크롬 드라이버 설정 및 실행
# ====================================================
def set_chrome_driver():
    chrome_options = Options()
    # chrome_options.add_argument("--headless")  # 브라우저 창 없이 실행하려면 주석 해제
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    # 봇 탐지 방지용 User-Agent 설정
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    # webdriver_manager를 통해 자동으로 드라이버 설치 및 실행
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

# ====================================================
# 2. 기사 상세 내용 추출 함수 (핵심 로직)
# ====================================================
def extract_article_info(driver, url):
    try:
        driver.get(url)
        time.sleep(0.5) # 페이지 로딩 및 리다이렉트 대기

        # [필터링] 연예 기사(entertain.naver.com) 여부 확인
        # 리스트에서는 안 보이다가 접속 후 리다이렉트 되는 경우를 잡습니다.
        if "entertain.naver.com" in driver.current_url:
            print(f" -> [패스] 연예 기사로 확인되어 건너뜁니다.")
            return None

        # [대기] 제목 요소가 뜰 때까지 최대 3초 대기
        try:
            WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "#title_area > span"))
            )
        except:
            # 타임아웃 시 다시 한번 URL 확인 (늦은 리다이렉트 대비)
            if "entertain.naver.com" in driver.current_url:
                print(f" -> [패스] 연예 기사(로딩 후 확인)로 건너뜁니다.")
                return None
            else:
                print(f" -> [스킵] 기사 형식이 다르거나 로딩 실패: {url}")
                return None

        # 1) 제목 추출
        try:
            title = driver.find_element(By.CSS_SELECTOR, "#title_area > span").text
        except:
            title = "제목 없음"

        # 2) 본문 추출 (이미지 캡션, 요약봇 등 불필요 요소 제거)
        try:
            dic_area = driver.find_element(By.CSS_SELECTOR, "#dic_area")
            
            # 자바스크립트로 불필요한 클래스(img_desc, media_end_summary) 삭제
            driver.execute_script("""
                var element = arguments[0];
                var dirts = element.querySelectorAll(".img_desc, .media_end_summary"); 
                for (var i = 0; i < dirts.length; i++) {
                    dirts[i].remove();
                }
            """, dic_area)
            
            content = dic_area.text
            content = content.replace("\n", " ").strip() # 줄바꿈을 공백으로 변경
        except:
            content = "본문 없음"

        # 3) 게재 시간 추출
        try:
            # 일반적인 네이버 뉴스 날짜 경로
            date_element = driver.find_element(By.CSS_SELECTOR, 
                "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span"
            )
            date_text = date_element.text
        except:
            # 경로가 다를 경우 예비 경로 시도
            try:
                date_text = driver.find_element(By.CSS_SELECTOR, ".media_end_head_info_datestamp span").text
            except:
                date_text = "날짜 없음"

        print(f"[수집 성공] {title} ({date_text})")
        
        # 수집된 데이터를 딕셔너리로 반환
        return {
            "제목": title, 
            "본문": content, 
            "날짜": date_text, 
            "링크": url
        }

    except Exception as e:
        print(f"[에러] 상세 페이지 수집 중 오류: {e}")
        return None

# ====================================================
# 3. 메인 실행 함수
# ====================================================
def main():
    driver = set_chrome_driver()
    driver.get(TARGET_URL)
    time.sleep(2)

    collected_data = [] # 결과 데이터를 담을 리스트
    page_count = 1      # 페이지 카운터

    try:
        while True:
            print(f"\n===== [ {page_count} 페이지 탐색 중 ] =====")
            
            # --- [Step A] 현재 리스트 페이지의 기사 URL 수집 ---
            article_urls = []
            
            # 1. 썸네일형 리스트 (ul.type06_headline)
            # 2. 일반 리스트 (ul.type06)
            # 공통: dt:not(.photo) > a 태그를 찾아야 이미지 링크 중복을 피함
            selectors = [
                "#main_content > div.list_body.newsflash_body > ul.type06_headline > li dl > dt:not(.photo) > a",
                "#main_content > div.list_body.newsflash_body > ul.type06 > li dl > dt:not(.photo) > a"
            ]

            for sel in selectors:
                links = driver.find_elements(By.CSS_SELECTOR, sel)
                for link in links:
                    url = link.get_attribute("href")
                    if url:
                        article_urls.append(url)
            
            print(f"-> 발견된 기사 링크: {len(article_urls)}개")

            # --- [Step B] 각 기사 상세 페이지 방문 및 데이터 추출 ---
            list_window = driver.current_window_handle # 현재 리스트 탭 저장
            
            for url in article_urls:
                # [사전 필터링] URL 자체에 entertain이 있으면 아예 안 들어감
                if "entertain.naver.com" in url:
                    continue

                # 새 탭 열기 (리스트 페이지 보호)
                driver.execute_script("window.open('');")
                driver.switch_to.window(driver.window_handles[-1])
                
                # 데이터 수집 함수 호출
                data = extract_article_info(driver, url)
                if data:
                    collected_data.append(data)
                
                # 탭 닫고 리스트 페이지로 복귀
                driver.close()
                driver.switch_to.window(list_window)
            
            # --- [Step C] 페이징(다음 페이지 이동) 처리 ---
            try:
                paging_area = driver.find_element(By.CSS_SELECTOR, "#main_content > div.paging")
                
                # 현재 페이지 번호 확인
                current_page_elem = paging_area.find_element(By.TAG_NAME, "strong")
                current_page_num = int(current_page_elem.text)
                
                # 이동할 페이지 번호 (현재 + 1)
                next_page_num = current_page_num + 1
                
                # 1. 숫자 버튼 찾기
                next_page_btn = None
                try:
                    next_page_btn = paging_area.find_element(By.XPATH, f".//a[text()='{next_page_num}']")
                except:
                    next_page_btn = None
                
                if next_page_btn:
                    print(f"-> {next_page_num} 페이지로 이동합니다.")
                    next_page_btn.click()
                    page_count += 1
                    time.sleep(2)
                    continue # 다음 루프로

                # 2. 숫자 버튼이 없으면 '다음(Next)' 화살표 버튼 찾기
                try:
                    next_group_btn = paging_area.find_element(By.CSS_SELECTOR, "a.next")
                    print("-> '다음' 그룹(Next) 버튼을 클릭합니다.")
                    next_group_btn.click()
                    page_count += 1
                    time.sleep(2)
                    continue
                except:
                    print("-> 더 이상 다음 페이지가 없습니다. 수집을 종료합니다.")
                    break

            except Exception as e:
                print(f"-> 페이징 처리 중 에러 혹은 마지막 페이지 도달: {e}")
                break
                
    except KeyboardInterrupt:
        print("\n[사용자 중단] 작업을 강제로 중단합니다.")
    
    finally:
        driver.quit() # 브라우저 종료
        
        # ====================================================
        # 4. 결과 저장 (CSV)
        # ====================================================
        print("\n[저장 시작] 데이터를 CSV 파일로 저장합니다...")
        
        if collected_data:
            df = pd.DataFrame(collected_data)
            
            # 파일명 생성 로직 (URL의 oid, date 파라미터 활용)
            try:
                if "oid=" in TARGET_URL:
                    # 예: ...&oid=009&date=20251219... -> oid=009&date=20251219.csv
                    # 파라미터 파싱을 더 정교하게 하기 위해 split 활용
                    params = TARGET_URL.split('?')[1]
                    oid = [p for p in params.split('&') if 'oid=' in p][0]
                    date_val = [p for p in params.split('&') if 'date=' in p][0]
                    filename = f"{oid}&{date_val}.csv"
                else:
                    raise ValueError
            except:
                # 실패 시 시간 기반 파일명 사용
                now = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"naver_news_result_{now}.csv"
            
            # utf-8-sig: 엑셀 한글 깨짐 방지
            df.to_csv(filename, index=False, encoding='utf-8-sig')
            
            print(f"✅ 저장 완료! 파일명: {filename}")
            print(f"📊 총 {len(df)}개의 기사가 수집되었습니다.")
        else:
            print("❌ 수집된 데이터가 없습니다.")

if __name__ == "__main__":
    main()


===== [ 1 페이지 탐색 중 ] =====
-> 발견된 기사 링크: 20개
[수집 성공] '양은이파' 조양은, 폭행사건 무죄 확정…435만원 국가 보상받는다 (2025.12.17. 오후 11:53)
[수집 성공] 연준 월러 이사 "추가 금리인하 지지…서두를 필요는 없어" (2025.12.17. 오후 11:41)
[수집 성공] 최중경 前장관, IFRS재단 이사 선임 (2025.12.17. 오후 11:36)
[수집 성공] 美기업 CFO들 "내년 소비자물가 약 4.2%"예상 (2025.12.17. 오후 11:26)
[수집 성공] 세계 최대 '19금 사이트' 회원정보 유출…"비트코인 안 주면 공개" (2025.12.17. 오후 11:22)
[수집 성공] '10배 레버리지 예측시장' 스페이스…토큰 공모 추진 (2025.12.17. 오후 11:18)
 -> [패스] 연예 기사로 확인되어 건너뜁니다.
 -> [패스] 연예 기사로 확인되어 건너뜁니다.
 -> [패스] 연예 기사로 확인되어 건너뜁니다.
 -> [패스] 연예 기사로 확인되어 건너뜁니다.
 -> [패스] 연예 기사로 확인되어 건너뜁니다.
 -> [패스] 연예 기사로 확인되어 건너뜁니다.
 -> [패스] 연예 기사로 확인되어 건너뜁니다.
 -> [패스] 연예 기사로 확인되어 건너뜁니다.
[수집 성공] 日 연구소 "서울, 도시경쟁력 세계 6위"…1위는 어디? (2025.12.17. 오후 10:42)
[수집 성공] 삼전·하이닉스 개미들 '잠 못 자겠네'…중대 발표에 '촉각' [종목+] (2025.12.17. 오후 10:01)
[수집 성공] "美·EU 전기차 정책 후퇴로 中 업체가 주도권 빼앗아 갈 것" (2025.12.17. 오후 9:49)
[수집 성공] '좀비담배' 확산에 말레이 칼 뽑았다…전자담배 전면금지 추진 (2025.12.17. 오후 9:43)
[수집 성공] 바이낸스 월렛, 인프라레드파이낸스(IR) TGE 진행..."4만명 참여" (2025.12.17. 오후 9:37)
[수집 성공] 베이징 북한 음식점

[수집 성공] 워너, 넷플릭스 선택…파라마운트 인수안 '퇴짜' (2025.12.17. 오후 5:38)
[수집 성공] 美의료유통사 메드라인, IPO로 63억달러 조달 (2025.12.17. 오후 5:37)
[수집 성공] '美주택 가늠자' 레나…실적 전망 기대 밑돌아 (2025.12.17. 오후 5:37)
[수집 성공] AI거품론에도…"펀드매니저 현금보유 사상 최저" (2025.12.17. 오후 5:37)
[수집 성공] "내일 봬요 누나" 뛰어넘었다…환승연애4, 티빙 구독자 '쑥' (2025.12.17. 오후 5:37)
[수집 성공] 두산그룹, SK실트론 품는다…반도체 전·후 공정 완성 (2025.12.17. 오후 5:37)
[수집 성공] 한동대, 신임 총장에 박성진 포스텍 교수 선임 (2025.12.17. 오후 5:36)
[수집 성공] 마트 돌며 빵에 '칼날' 넣은 美 여성 엽기 행각 (2025.12.17. 오후 5:35)
-> 8 페이지로 이동합니다.

===== [ 8 페이지 탐색 중 ] =====
-> 발견된 기사 링크: 20개
[수집 성공] 쿠팡, 4조 '과징금 폭탄' 위기…美선 '늑장 공시' 집단소송 임박 (2025.12.17. 오후 5:34)
[수집 성공] 구체적 보상안 안 밝힌 쿠팡…과방위 "불출석 증인들 고발" (2025.12.17. 오후 5:33)
[수집 성공] "슈드보다 쏠쏠"…빅테크 품은 커버드콜에 돈 몰린다 (2025.12.17. 오후 5:33)
[수집 성공] [포토] 원·달러 환율 8개월 만에 장중 1480원 돌파 (2025.12.17. 오후 5:33)
[수집 성공] '부실 징후' 대기업 6곳 늘어…부동산·자동차업종 재무구조 악화 (2025.12.17. 오후 5:32)
[수집 성공] 의결권 자문사 압박하는 트럼프…국내 기업 '주총 나비효과' 촉각 (2025.12.17. 오후 5:32)
[수집 성공] 이재명 대통령 한마디에 주가 널뛰는 테마주 (2025.12.17. 오후 5:32)
[수집 성공] 재취업 엄격한 방사청 출신, 방산기업 속속 이

[수집 성공] 하나·신한 발행어음 인가…삼성·메리츠는 아직, 종투사 7곳으로 (2025.12.17. 오후 4:01)
[수집 성공] [마켓칼럼] 내년에도 국내 증시 활황세…"전력기기 K뷰티 유망" (2025.12.17. 오후 4:01)
[수집 성공] 中企 해외판로 개척, K푸드 싱가포르 진출…동반성장 4년째 '최우수' 받은 인천항만公 (2025.12.17. 오후 4:01)
-> 14 페이지로 이동합니다.

===== [ 14 페이지 탐색 중 ] =====
-> 발견된 기사 링크: 20개
[수집 성공] 장애인 일자리, 기부형 축제, 지역생태보호…공공 MICE기관의 역할 새롭게 쓰는 킨텍스 (2025.12.17. 오후 4:00)
[수집 성공] 집값 계속 깎아주더니…'워런 버핏이 찜' 美 건설주, 수익성 확 깎였다 (2025.12.17. 오후 3:59)
[수집 성공] 경기도경제과학진흥원, 파주 이전 공식화 (2025.12.17. 오후 3:58)
[수집 성공] 전통시장 위생교육, 9년째 장학금…사회공헌 외연 넓히는 신세계 (2025.12.17. 오후 3:58)
[수집 성공] 종이 인쇄물 감축, MICE 인재양성…'실천형 ESG 원년' 선포한 엑스코 (2025.12.17. 오후 3:57)
[수집 성공] 한국P&G, 연말 회식·홈파티 뒤 코트·소파 냄새…페브리즈로 한꺼번에 싹~ 없애요 (2025.12.17. 오후 3:56)
[수집 성공] 특별한 사람과 소중한 시간…연말엔 '경험'을 선물하세요 (2025.12.17. 오후 3:56)
[수집 성공] 에스.티.듀퐁, 153년 전통 프랑스 명품…'품격'을 선물하세요 (2025.12.17. 오후 3:55)
[수집 성공] 반얀트리 서울, 충남 논산 '킹스베리'로 꽉 채웠다…그릇까지 품격있는 디저트 뷔페 (2025.12.17. 오후 3:55)
[수집 성공] 우유자조금관리위원회 "가격보다 품질"…국내생산·빠른 유통 믿고 '신선우유' 찾는다 (2025.12.17. 오후 3:55)
[수집 성공] 오뚜기, 단순 파스타 소스 넘어 '유러피언 식

[수집 성공] "좋은 생각"…李 대통령 극찬한 보고 뭐길래 (2025.12.17. 오후 1:14)
 -> [패스] 연예 기사로 확인되어 건너뜁니다.
 -> [패스] 연예 기사로 확인되어 건너뜁니다.
 -> [패스] 연예 기사로 확인되어 건너뜁니다.
-> 20 페이지로 이동합니다.

===== [ 20 페이지 탐색 중 ] =====
-> 발견된 기사 링크: 20개
[수집 성공] '골칫거리' 스팸, 제대로 막는다…무효번호 사전 탐지 시작 (2025.12.17. 오후 1:09)
 -> [패스] 연예 기사로 확인되어 건너뜁니다.
 -> [패스] 연예 기사로 확인되어 건너뜁니다.
 -> [패스] 연예 기사로 확인되어 건너뜁니다.
[수집 성공] '도시 경쟁력' 세계 1위는 런던…서울은 몇 위인가 보니 (2025.12.17. 오후 12:59)
[수집 성공] '코메론' 52주 신고가 경신, 줄자 생산 기업, Valuation 저평가 (2025.12.17. 오후 12:55)
 -> [패스] 연예 기사로 확인되어 건너뜁니다.
 -> [패스] 연예 기사로 확인되어 건너뜁니다.
[수집 성공] 환율, 외국인 매도·달러 강세에 장중 1480원 돌파…8개월 만에 최고 (2025.12.17. 오후 12:45)
[수집 성공] 로저스 쿠팡 대표 "개인 정보유출, 美SEC 의무 보고 사항 아냐" (2025.12.17. 오후 12:35)
[수집 성공] 의협 "탈모 건보 적용 우선순위 맞나…재정은 중증 질환에 써야" (2025.12.17. 오후 12:18)
[수집 성공] '다이소 대항마' 깜짝 등장…'5000원 이하' 제품만 내놓는 곳 (2025.12.17. 오후 12:18)
[수집 성공] [속보] 김용범 정책실장 "광주 군공항 이전 합의 이뤄" (2025.12.17. 오후 12:10)
[수집 성공] [속보] 불법 부패 외 정책감사 폐지 명문화…감사원, 李대통령 지시 이행 (2025.12.17. 오후 12:07)
[수집 성공] '재벌돌' 애니 할머니 이명희 회장 자택…공시가격 11년 연속 1위 (


===== [ 26 페이지 탐색 중 ] =====
-> 발견된 기사 링크: 20개
[수집 성공] [속보] 김건희특검, '로저비비에 선물' 김기현 자택 압수수색 (2025.12.17. 오전 9:23)
[수집 성공] 에어프랑스, 중거리 노선 확대…파리 샤를드골-런던 개트윅 잇는다 (2025.12.17. 오전 9:23)
[수집 성공] 홍콩계 거캐피탈, 폐기물 업체 코엔텍 인수 (2025.12.17. 오전 9:21)
[수집 성공] '피에이치에이' 52주 신고가 경신, 외국인 5일 연속 순매수(2.5만주) (2025.12.17. 오전 9:19)
[수집 성공] '아이에스동서' 52주 신고가 경신, 긍정적인 신호 2가지 - BNK투자증권, 매수 (2025.12.17. 오전 9:19)
 -> [패스] 연예 기사로 확인되어 건너뜁니다.
[수집 성공] [포토] 홈플러스, 대한민국 대표 연말 세일 '윈터 홈플런' 시작 (2025.12.17. 오전 9:17)
[수집 성공] '금호건설' 52주 신고가 경신, 광명시흥 A2-5BL, A1-1BL 및 B1-7BL 통합형 민간참여 공공주택건설사업 1,627.4억원 (매출액대비 8.50 %) (2025.12.17. 오전 9:16)
[수집 성공] '원바이오젠' 52주 신고가 경신, 외국인, 기관 각각 5일, 7일 연속 순매수 (2025.12.17. 오전 9:16)
[수집 성공] "특혜" 한마디에 주가 폭락…李 대통령 입에 개미들 '벌벌' [종목+] (2025.12.17. 오전 9:16)
[수집 성공] '현대위아' 52주 신고가 경신, 최근 3일간 기관 대량 순매수 (2025.12.17. 오전 9:14)
[수집 성공] 역대 최대 규모 '코리아그랜드세일 개막'…"방한 관광 촉진·내수 활성화" (2025.12.17. 오전 9:14)
[수집 성공] "아이 학원비를 줄일 순 없고"…편의점서 끼니 때우는 4050 ['영포티' 세대전쟁] (2025.12.17. 오전 9:12)
[수집 성공] 나라스페이스테크놀로지, 코스닥 상장 첫날 장 초반 '따블' (2